In [ ]:
!pip install "huggingface_hub[cli]" datasets transformers trl torch wandb ipywidgets ipykernel iprogress

In [2]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: embereagle to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
from datasets import load_dataset

# https://huggingface.co/datasets/HuggingFaceTB/smoltalk
dataset = load_dataset("HuggingFaceTB/smoltalk", 'all')

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['messages', 'source'],
        num_rows: 1043917
    })
    test: Dataset({
        features: ['messages', 'source'],
        num_rows: 54948
    })
})

In [6]:
dataset['train'][0]

{'messages': [{'content': 'The function \\( g(x) \\) satisfies the functional equation\n\\[ g(x + y) = g(x) + g(y) \\]\nfor all real numbers \\( x \\) and \\( y \\), and it is given that \\( g(3) = 4 \\). Find \\( g(10) \\).',
   'role': 'user'},
  {'content': 'Given the functional equation and the specific value \\( g(3) = 4 \\), we can find \\( g(1) \\) by using the equation multiple times:\n\\[\ng(3) = g(2) + g(1)\n\\]\n\\[\ng(2) = g(1) + g(1) = 2g(1)\n\\]\nThus,\n\\[\n4 = 2g(1) + g(1) = 3g(1)\n\\]\n\\[\ng(1) = \\frac{4}{3}\n\\]\nNow we can find \\( g(10) \\) using \\( g(1) \\):\n\\[\ng(10) = 10g(1) = 10 \\times \\frac{4}{3} = \\frac{40}{3}\n\\]\nHence, the value of \\( g(10) \\) is \\(\\boxed{\\frac{40}{3}}\\).',
   'role': 'assistant'}],
 'source': 'numina-cot-100k'}

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# https://huggingface.co/microsoft/DialoGPT-medium
# model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
# tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# https://huggingface.co/Qwen/Qwen2.5-0.5B
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B", use_fast=True)

# https://huggingface.co/distilbert/distilgpt2
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2", use_fast=True)

In [8]:
import multiprocessing
num_proc = multiprocessing.cpu_count()

def chatml_tokenize(batch):
    texts = []
    for messages in batch["messages"]:
        chat = ""
        for msg in messages:
            if msg["role"] == "user":
                chat += "<|user|> " + msg["content"].strip() + " " + tokenizer.eos_token + " "
            elif msg["role"] == "assistant":
                chat += "<|assistant|> " + msg["content"].strip() + " " + tokenizer.eos_token + " "
        texts.append(chat.strip())
    return tokenizer(texts, padding=False, truncation=False)

tokenized_train = dataset["train"].map(
    chatml_tokenize, batched=True, batch_size=1000, num_proc=num_proc, remove_columns=["messages"]
)
tokenized_test = dataset["test"].map(
    chatml_tokenize, batched=True, batch_size=1000, num_proc=num_proc, remove_columns=["messages"]
)

In [ ]:
# Should show ChatML-formatted text
print("Sample training example:")
print(tokenizer.decode(tokenized_train[0]["input_ids"])) 
# Expected: "<|user|> ... <|assistant|> ..."

In [9]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print(device)

cuda


In [ ]:
# Add special tokens to tokenizer
special_tokens = ["<|user|>", "<|assistant|>"]
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Evaluate WITHOUT ChatML formatting
def base_model_eval(question):
    encoded = tokenizer(question, return_tensors="pt").to(device)
    generated = model.generate(**encoded, max_new_tokens=20)
    return tokenizer.decode(generated[0], skip_special_tokens=True)

print("BEFORE TRAINING (Raw model):")
print(base_model_eval("The capital of France is"))
print(base_model_eval("What is the capital of France?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The capital of France is ÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂ
What is the capital of France?






















In [11]:
import random

# sample random indices from the test set
random_indices = random.sample(range(len(tokenized_test)), 400)

# create a new Dataset with only those
sampled_eval_dataset = tokenized_test.select(random_indices)

In [ ]:
from transformers import DataCollatorForLanguageModeling
from trl import SFTConfig, SFTTrainer

# Enable gradient checkpointing to reduce memory
# model.gradient_checkpointing_enable()
# model.config.use_cache = False

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding if not set

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# training on lightning studio cloud L4 GPU
training_args = SFTConfig(
    output_dir="./trainer_output",
    per_device_train_batch_size=16,
    max_steps=1000,
    learning_rate=1e-5,
    bf16=True,
    logging_steps=10,
    save_total_limit=2,  # Keep last 2 checkpoints
    save_strategy="steps",
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    optim="adamw_torch_fused",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    run_name="lightning_l4"
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=sampled_eval_dataset,
    data_collator=data_collator,
)

trainer.train()


Converting eval dataset to ChatML:   0%|          | 0/400 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,2.531700,2.425086
100,2.537700,2.362699
150,2.431300,2.331207
200,2.468000,2.312342
250,2.394100,2.291852
300,2.424900,2.273607
350,2.419900,2.261746
400,2.271300,2.252806
450,2.454800,2.242353
500,2.246500,2.234940


TrainOutput(global_step=1000, training_loss=2.3915605659484864, metrics={'train_runtime': 1681.3652, 'train_samples_per_second': 9.516, 'train_steps_per_second': 0.595, 'total_flos': 4180748009472000.0, 'train_loss': 2.3915605659484864})

In [ ]:
# Evaluate WITH ChatML formatting
def chatml_eval(question):
    formatted_prompt = f"<|user|> {question} <|assistant|>"
    encoded = tokenizer(formatted_prompt, return_tensors="pt").to(device)
    generated = model.generate(**encoded, max_new_tokens=20)
    full = tokenizer.decode(generated[0], skip_special_tokens=False)
    return full.split("<|assistant|>")[-1].replace(tokenizer.eos_token, "").strip()

print("\nAFTER TRAINING (ChatML-formatted):")
print(chatml_eval("The capital of France is"))
print(chatml_eval("What is the capital of France?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The capital of France is                     
What is the capital of France? 

The answer is yes. 

The answer is yes. 

The


In [ ]:
assert model.get_output_embeddings().weight.data_ptr() == model.get_input_embeddings().weight.data_ptr()

In [ ]:
# Continue training, use last checkpoint
trainer.args.max_steps += 1000
trainer.train(resume_from_checkpoint=True)

In [ ]:
# Evaluate WITH ChatML formatting
def chatml_eval(question):
    formatted_prompt = f"<|user|> {question} <|assistant|>"
    encoded = tokenizer(formatted_prompt, return_tensors="pt").to(device)
    generated = model.generate(**encoded, max_new_tokens=20)
    full = tokenizer.decode(generated[0], skip_special_tokens=False)
    return full.split("<|assistant|>")[-1].replace(tokenizer.eos_token, "").strip()

print("\nAFTER TRAINING (ChatML-formatted):")
print(chatml_eval("The capital of France is"))
print(chatml_eval("What is the capital of France?"))